<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/Multi_Kernel_1D_CNN_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#GPU사용
import torch
import torchvision
import torch.nn

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [4]:
#"""이전것 import부분 tf.으로 다 바꿔줌"""#
#""" from keras.models import Sequential"""
#""" from keras import layers"""
#""" from keras.optimizers import RMSprop"""

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
# with tf.device('/device:GPU:0'):에서 모델 돌리기

In [5]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.2 MB/s eta 0:00:00


In [6]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7b72d80c7730>)

In [8]:
train=pd.read_table('ratings_train.txt')
test=pd.read_table('ratings_test.txt')

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [10]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
train=train.drop(['id'], axis=1)

In [12]:
train['document']=train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '')
train['document'].replace('',np.nan,inplace=True)
train.drop_duplicates(subset=['document'], inplace=True)
train=train.dropna(how='any')

<ipython-input-12-7a8bf1c43fe5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['document']=train['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '')


In [13]:
test['document']=test['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '')
test['document'].replace('',np.nan,inplace=True)
test.drop_duplicates(subset=['document'], inplace=True)
test=test.dropna(how='any')

<ipython-input-13-2b1b7ed8abf6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test['document']=test['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '')


In [14]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [15]:
okt=Okt()

In [17]:
X_train=[]
with tf.device('device:GPU:0'):
  for sentence in tqdm(train['document']):
    sentence=okt.morphs(sentence, stem=True)
    stopwords_removed_sentence=[word for word in sentence if not word in stopwords]
    X_train.append(stopwords_removed_sentence)

100%|██████████| 142878/142878 [1:30:12<00:00, 26.40it/s]


In [18]:
with tf.device('device:GPU:0'):
  X_test=[]
  for sentence in tqdm(test['document']):
    sentence=okt.morphs(sentence, stem=True)
    stopwords_removed_sentence=[word for word in sentence if not word in stopwords]
    X_test.append(stopwords_removed_sentence)

100%|██████████| 48210/48210 [32:49<00:00, 24.48it/s]


In [19]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train)

In [20]:
threshold=3

total_cnt=len(tokenizer.word_index)
total_freq=0
rare_freq=0
rare_cnt=0

for key, value in tokenizer.word_counts.items():
  total_freq+=value
  if value<threshold:
    rare_freq+=value
    rare_cnt+=1
print("유니크한 단어의 비율 : " ,rare_cnt/total_cnt)
print('유니크한 단어 빈도의 비율 : ',rare_freq/total_freq)

유니크한 단어의 비율 :  0.5479861514310904
유니크한 단어 빈도의 비율 :  0.01983405061632583


In [21]:
vocab_size=total_cnt-rare_cnt+1
tokenizer=Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)
y_train=np.array(train['label'])
y_test=np.array(test['label'])

In [22]:
drop_train=[index for index, sentence in enumerate(X_train)if len(sentence)<1]

X_train=np.delete(X_train, drop_train, axis=0)
y_train=np.delete(y_train, drop_train, axis=0)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:5030: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [23]:
def below_threshold_len(max_len, nested_list):
  count=0
  for sentence in nested_list:
    if len(sentence)<=max_len:
      count+=1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율 : %s' %(max_len, (count/len(nested_list))*100))

In [24]:
max_len=30
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 30 이하인 샘플의 비율 : 94.29964228098477


In [25]:
X_train=pad_sequences(X_train, maxlen=max_len)
X_test=pad_sequences(X_test, maxlen=max_len)

In [28]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim=128
dropout_ratio=(0.5, 0.8)
num_filters=128
hidden_units=128

In [29]:
model_input=Input(shape=(max_len, )) #max_len만큼의 벡터의 크기를 가지고 들어온다.
z=Embedding(vocab_size, embedding_dim, input_length=max_len, name="embedding")(model_input)#vocab_size로 들어오고 embedding_dim크기로 나간다. max_len만큼 반복한다.
z=Dropout(dropout_ratio[0])(z)

In [31]:
conv_blocks=[]

for sz in [3,4,5]: #커널의 크기 3, 4, 5
  conv=Conv1D(filters=num_filters, kernel_size=sz, padding='valid', activation='relu', strides=1)(z)
  conv=GlobalMaxPooling1D()(conv)
  conv_blocks.append(conv)

In [32]:
z=Concatenate()(conv_blocks) if len(conv_blocks)>1 else conv_blocks[0]
z=Dropout(dropout_ratio[1])(z)
z=Dense(hidden_units, activation='relu')(z)
model_output=Dense(1, activation='sigmoid')(z)

model=Model(model_input, model_output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
with tf.device('device:GPU:0'):
  model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=64, verbose=2, callbacks=[es, mc])

Epoch 1/10

Epoch 1: val_acc improved from -inf to 0.83661, saving model to best_model.h5
1783/1783 - 77s - loss: 0.4442 - acc: 0.7836 - val_loss: 0.3797 - val_acc: 0.8366 - 77s/epoch - 43ms/step
Epoch 2/10

Epoch 2: val_acc improved from 0.83661 to 0.84264, saving model to best_model.h5
1783/1783 - 17s - loss: 0.3531 - acc: 0.8475 - val_loss: 0.3651 - val_acc: 0.8426 - 17s/epoch - 10ms/step
Epoch 3/10

Epoch 3: val_acc did not improve from 0.84264
1783/1783 - 15s - loss: 0.3182 - acc: 0.8658 - val_loss: 0.3630 - val_acc: 0.8397 - 15s/epoch - 8ms/step
Epoch 4/10

Epoch 4: val_acc did not improve from 0.84264
1783/1783 - 14s - loss: 0.2918 - acc: 0.8791 - val_loss: 0.3675 - val_acc: 0.8406 - 14s/epoch - 8ms/step
Epoch 5/10

Epoch 5: val_acc did not improve from 0.84264
1783/1783 - 16s - loss: 0.2700 - acc: 0.8889 - val_loss: 0.3744 - val_acc: 0.8383 - 16s/epoch - 9ms/step
Epoch 6/10

Epoch 6: val_acc did not improve from 0.84264
1783/1783 - 15s - loss: 0.2501 - acc: 0.8973 - val_loss: 0

In [33]:
loaded_model=load_model('best_model.h5')
print("\n 테스트 정확도 : %.4f"%(loaded_model.evaluate(X_test, y_test)[1]))

1507/1507 [==============================] - 6s 4ms/step - loss: 0.3710 - acc: 0.8374

 테스트 정확도 : 0.8374


In [34]:
def sentiment_predict(new_sentence):
  new_sentence=re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', new_sentence)
  new_sentence=okt.morphs(new_sentence, stem=True)
  new_sentence=[word for word in new_sentence if not word in stopwords]
  encoded=tokenizer.texts_to_sequences([new_sentence])
  paded=pad_sequences(encoded, maxlen=max_len)
  score=float(loaded_model.predict(paded))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [35]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

1/1 [==============================] - 0s 222ms/step
99.25% 확률로 긍정 리뷰입니다.



In [36]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

1/1 [==============================] - 0s 20ms/step
93.34% 확률로 부정 리뷰입니다.



In [37]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

1/1 [==============================] - 0s 19ms/step
96.28% 확률로 부정 리뷰입니다.



In [38]:
sentiment_predict('감독 뭐하는 놈이냐?')

1/1 [==============================] - 0s 22ms/step
87.95% 확률로 부정 리뷰입니다.



In [39]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

1/1 [==============================] - 0s 19ms/step
65.93% 확률로 긍정 리뷰입니다.

